### Authenticate

The Gemini API lets you tune models on your own data. Since it's your data and
your tuned models this needs stricter access controls than API-Keys can provide.

Before you can run this tutorial, you'll need to
[setup OAuth for your project](https://ai.google.dev/gemini-api/docs/oauth).


In Colab the easiest wat to get setup is to copy the contents of your `client_secret.json` file into Colab's "Secrets manager" (under the key icon in the left panel) with the secret name `CLIENT_SECRET`.

This gcloud command turns the `client_secret.json` file into credentials that can be used to authenticate with the service.

> Important: If you're running this in Colab, **don't just click the link it prints**. That will fail. Follow the instructions and copy the `gcloud` command it prints to your local machine and run it there, then paste the output from your local machine back here.


In [ ]:
import os
if 'COLAB_RELEASE_TAG' in os.environ:
  from google.colab import userdata
  import pathlib
  pathlib.Path('client_secret.json').write_text(userdata.get('CLIENT_SECRET'))

  # Use `--no-browser` in colab
  !gcloud auth application-default login --no-browser --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'
else:
  !gcloud auth application-default login --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'

### Install the client library

In [3]:
!pip install -q google-generativeai

### Import libraries

In [4]:
import google.generativeai as genai

You can check you existing tuned models with the `genai.list_tuned_model` method.

In [ ]:
for i, m in zip(range(5), genai.list_tuned_models()):
  print(m.name)

### NER for preprocessing the news to recover the location

In [ ]:
!python -m spacy download es_core_news_lg

In [7]:
import spacy

In [8]:
nlp = spacy.load("es_core_news_lg")

In [9]:
def extract_first_loc(text):
    doc = nlp(text)
    first_loc = next((ent.text for ent in doc.ents if ent.label_ == "LOC"), None) # Find the first entity of type LOC

    return first_loc

### Processing the news and generation function

In [19]:
def generar_nueva_noticia_gemini(noticia, fecha):

    system_message = 'Tu tarea es escribir artículos de noticia que contengan siempre una fecha, un lugar y un acontecimiento. No puedes inventar información que no se te da, utiliza lenguaje formal.'
    loc = extract_first_loc(noticia)

    if(loc):
            prompt = system_message + "Crea un artículo de noticias con esta información: " + noticia + ". " + "Fecha: " + fecha + "." + "Lugar: " + loc + "."
    else:
            prompt = system_message + "Crea un artículo de noticias con esta información: " + noticia + ". " + "Fecha: " + fecha

    model = genai.GenerativeModel(model_name=m.name)

    try:
        response = model.generate_content(prompt)
        nueva_noticia = response.text

        return nueva_noticia

    except Exception as e:

        print(f"Error al generar nueva noticia con Google: {e}")
        return ""

### Generate the news

In [14]:
# Give information about the news. This is an example!
noticia = "Melbourne. El español Carlos Sainz (Ferrari), ganador del Gran Premio de Australia de Fórmula Uno, aseguró que confiaba en tener la capacidad de superar al tricampeón mundial y actual líder de la competencia, el neerlandés Max Verstappen (Red Bull), hoy en la tercera prueba de la temporada, e indicó que su triunfo demuestra que nunca hay que darse por vencido."
fecha = "25 de marzo del 2024"

In [ ]:
print(generar_nueva_noticia_gemini(noticia, fecha))